# Counting



The **Quantum Counting** quantum kata is a series of exercises designed
to get you familiar with quantum counting algorithm.

It covers the following topics:

* writing oracles for counting,
* performing actual counting.
    

*Reading material:*

* The tasks follow the explanation from *Quantum Computation and Quantum Information* by Nielsen and Chuang.
  In the 10th anniversary edition, this is section 6.3 on pages 261-263.
  
Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blanks (marked with the `// ...` comments)
with some Q# code that solves the task.  To verify your answer, run the cell with Ctrl/⌘+Enter.

Within each section, tasks are given in approximate order of increasing difficulty;
harder ones are marked with asterisks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.8.1907.1701


> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Oracle Counting


### Task 1.1. The Sprinkler oracle

Let us consider an example inspired by the sprinkler problem of (Pearl 1988): 
we have three Boolean variable, $s$, $r$, $w$ representing respectively propositions 
“the sprinkler was on”, "ıt rained last night” and “the grass is wet”. 
We know that if the sprinkler was on the grass is wet ($s \to w$), 
if it rained last night the grass is wet ($r \to w$) 
and that the the sprinkler being on and rain last night cannot be true at the same time ($s, r \to$).
Transformed in conjunctive normal formal we obtain formula $(\neg s \vee w) \wedge (\neg r \vee w) \wedge
(\neg s \vee \neg r)$.
Let $s,r,w$=`queryRegister[0],queryRegister[1],queryRegister[2]`
Hint: to solve this task you also need to use ancilla qubits
This formula has 4 models out of 8 possible worlds.

<br/>
<details>
  <summary>Need a hint? Click here </summary>
    Here is what the circuit looks like
<img src="sprinkler.png" />
</details>


In [10]:
%kata T11_Oracle_Sprinkler_Test 


operation Oracle_Sprinkler (queryRegister : Qubit[], target : Qubit, ancilla : Qubit[]) : Unit
    {      
	       body (...) {
				X(queryRegister[2]);
				X(ancilla[0]);
				X(ancilla[1]);
				X(ancilla[2]);
                
				CCNOT(queryRegister[0],queryRegister[1],ancilla[0]);
				CCNOT(queryRegister[1],queryRegister[2],ancilla[1]);
				CCNOT(queryRegister[0],queryRegister[2],ancilla[2]);
				(Controlled X)([ancilla[0],ancilla[1],ancilla[2]],target);
				CCNOT(queryRegister[0],queryRegister[2],ancilla[2]);
				CCNOT(queryRegister[1],queryRegister[2],ancilla[1]);
				CCNOT(queryRegister[0],queryRegister[1],ancilla[0]);

				X(ancilla[2]);
				X(ancilla[1]);
				X(ancilla[0]);            
           }
			adjoint invert;
			controlled auto;
			controlled adjoint auto;
    }

/snippet:(1,26): error QS6104: No namespace with that name exists.
/snippet:(3,10): error QS6104: No namespace with that name exists.
/snippet:(14,5): error QS5022: No identifier with that name exists.
/snippet:(15,5): error QS5022: No identifier with that name exists.
/snippet:(16,5): error QS5022: No identifier with that name exists.
/snippet:(17,5): error QS5022: No identifier with that name exists.
/snippet:(19,5): error QS5022: No identifier with that name exists.
/snippet:(20,5): error QS5022: No identifier with that name exists.
/snippet:(21,5): error QS5022: No identifier with that name exists.
/snippet:(22,17): error QS5022: No identifier with that name exists.
/snippet:(23,5): error QS5022: No identifier with that name exists.
/snippet:(24,5): error QS5022: No identifier with that name exists.
/snippet:(25,5): error QS5022: No identifier with that name exists.
/snippet:(27,5): error QS5022: No identifier with that name exists.
/snippet:(28,5): error QS5022: No identifier with


## Part II. Counting

## Task 2.1. Implement counting using operations from ReferenceImplementation.qs
* UnitaryPowerImpl, for computing powers of a unitary operation
* GroverIteration for the Grover operator
* QuantumPhaseEstimation, for estimating the phase
* Counting should return the number of models, 4 in this case


In [6]:
   operation Counting() : Double {
        mutable phase = -1.0;
        let n=4;
        using ((reg,phaseRegister,ancilla)=(Qubit[3],Qubit[n],Qubit[3]))
                                           {
        // Construct a phase estimation oracle from the unitary
        let phaseOracle = OracleConverter(Oracle_Sprinkler_Reference(_,_,ancilla));

        let oracle = DiscreteOracle(UnitaryPowerImpl(GroverIteration(_, phaseOracle), _, _));


        // Allocate qubits to hold the eigenstate of U and the phase in a big endian register 
            
            let phaseRegisterBE = BigEndian(phaseRegister);
            // Prepare the eigenstate of U
                HadamardTransform(reg);
//should return 0.5
            // Call library
            QuantumPhaseEstimation(oracle, reg, phaseRegisterBE);
            // Read out the phase
            set phase = IntAsDouble(MeasureInteger(BigEndianAsLittleEndian(phaseRegisterBE))) / IntAsDouble(1 <<< (n));

            ResetAll(reg);
            ResetAll(phaseRegister);
        }
        let angle = PI()*phase;
        let res = (PowD(Sin(angle),2.0));

        return 8.0*res;
    }


/snippet:(1,26): error QS6104: No namespace with that name exists.
/snippet:(7,27): error QS5022: No identifier with that name exists.
/snippet:(7,43): error QS5022: No identifier with that name exists.
/snippet:(9,22): error QS5022: No identifier with that name exists.
/snippet:(9,37): error QS5022: No identifier with that name exists.
/snippet:(9,54): error QS5022: No identifier with that name exists.
/snippet:(14,35): error QS5022: No identifier with that name exists.
/snippet:(16,17): error QS5022: No identifier with that name exists.
/snippet:(19,13): error QS5022: No identifier with that name exists.
/snippet:(21,25): error QS5022: No identifier with that name exists.
/snippet:(21,37): error QS5022: No identifier with that name exists.
/snippet:(21,52): error QS5022: No identifier with that name exists.
/snippet:(21,97): error QS5022: No identifier with that name exists.
/snippet:(23,13): error QS5022: No identifier with that name exists.
/snippet:(24,13): error QS5022: No identi

### Task 3.2. Using Counting 

**Goal:**   Use your implementation of Counting from Task 2.1 and the oracle from part 1
  to find the number of solutions of the sprinkler problem. This task is not covered by a test and allows you to experiment with running the algorithm.
  
> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_Counting` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<br/>
<details closed>
  <summary>Hint #3</summary>
    You can use the Message function to output the results.
</details>

In [ ]:
   operation Run_Counting() : Unit {

		let res=Counting();
        Message(DoubleAsString(res));

    }


In [4]:
%simulate Run_Counting

Invalid operation name: Run_Counting
